<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text2Vec_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import nltk
import string
import sklearn
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import warnings
warnings.filterwarnings('ignore')

import gensim
import sklearn
nltk.download('stopwords')
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Doc2Vec

In [ ]:
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
!ls -lah DATAPATH
path = "DATAPATH/train_data.csv"

In [7]:
df = pd.read_csv(path)
df.shape

(30000, 2)

In [10]:
df.tail(10)

,sentiment,content
29990,fun,finally get my hands on my laptop! just ate t...
29991,happiness,@twtboxdj Thanks Mr. DJ!
29992,love,@Oprah Happy mother's day Oprah. You're a mom...
29993,neutral,Just finished watching He's Just Not That Into...
29994,fun,Hanging with my cousin Jimmy then hopefully ha...
29995,happiness,I had a great date last night...tried to find ...
29996,sadness,With alex
29997,happiness,@fureousangel that is comedy good luck my fri...
29998,fun,stephs grad party gr8! shoved cake in her face...
29999,happiness,@jesfive SWEEEEET - San Fran is awesome!!!! L...


In [12]:
df.sentiment.value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [17]:
selected_features = ['worry', 'neutral', 'happiness']
sub_df = df[df.sentiment.isin(selected_features)]
sub_df

,sentiment,content
4,neutral,@dannycastillo We want to trade with someone w...
5,worry,Re-pinging @ghostridah14: why didn't you go to...
7,worry,Hmmm. http://www.djhero.com/ is down
10,neutral,cant fall asleep
11,worry,Choked on her retainers
...,...,...
29991,happiness,@twtboxdj Thanks Mr. DJ!
29993,neutral,Just finished watching He's Just Not That Into...
29995,happiness,I had a great date last night...tried to find ...
29997,happiness,@fureousangel that is comedy good luck my fri...


In [24]:
tweeter = nltk.tokenize.TweetTokenizer(strip_handles=True, preserve_case=False)
stopwords = set(nltk.corpus.stopwords.words('english'))

stopwords_digits_remove = lambda tokens: [token for token in tokens if token not in stopwords and not token.isdigit()]
preprocess = lambda doc: [stopwords_digits_remove(tweeter.tokenize(content)) for content in doc]

mydata = preprocess(sub_df.content)
mycats = sub_df.sentiment

In [25]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(mydata, mycats, random_state=324)

In [29]:
train_doc2vec = [gensim.models.doc2vec.TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(x_train)]

In [30]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, alpha=0.025, dm=1, epochs=100)
model.build_vocab(train_doc2vec)

In [31]:
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)

In [35]:
train_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in x_train]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in x_test]

In [39]:
print(metrics.classification_report(y_test, preds))

              precision    recall  f1-score   support

   happiness       0.36      0.51      0.42       761
     neutral       0.48      0.57      0.52      1597
       worry       0.62      0.41      0.50      1832

    accuracy                           0.49      4190
   macro avg       0.49      0.50      0.48      4190
weighted avg       0.52      0.49      0.49      4190



In [44]:
train_vectors[0]

array([ 0.11641004, -0.04375543,  0.02039554,  0.25807506, -0.08165002,
        0.15623224,  0.02950598, -0.11502851,  0.00719432, -0.38904315,
       -0.02489494,  0.01830048,  0.02387852, -0.10338783, -0.09047322,
        0.15834954, -0.08306049, -0.09786002, -0.03367309, -0.06070329,
       -0.01772952, -0.11380818,  0.14539179,  0.06817325,  0.07645974,
       -0.09828415, -0.24917495,  0.1679408 , -0.20077914, -0.01163829,
       -0.09039664, -0.15442355, -0.02928317, -0.07261185,  0.18637472,
       -0.23207411, -0.03359758, -0.07133026,  0.04368724, -0.13428563,
        0.16382065, -0.13752255, -0.11907786,  0.12010764,  0.01540544,
       -0.0215659 , -0.21957673,  0.23439862, -0.2232042 , -0.2880198 ],
      dtype=float32)

In [37]:
lr_model = linear_model.LogisticRegression(class_weight='balanced')
lr_model.fit(train_vectors, y_train)
preds = lr_model.predict(test_vectors)

In [43]:
np.array(train_vectors).shape, np.array(test_vectors).shape

((12569, 50), (4190, 50))